# On utilisera le truc etude n° 10

In [25]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal


H=4
Vvec_moy = [[] for i in range(5)]
lettres = ['a', 'e', 'i', 'o', 'u']
a = range(5)

for lettre,j in zip(lettres, a):
    signaux = []
    for i in range(5):
        dat = np.load(f'data_partie_2/apprentissage/{lettre}_{i+1}.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
        s = dat['s'] # extraction du signal utile
        t = dat['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t)
        S = np.abs(S)

        s_centre= s - np.mean(s)
        nu, S_norm = san.trans_fourier(s_centre, t)
        S_norm = np.abs(S_norm)
        S_norm /= np.max(S_norm)

        f0 = san.detect_fondamentale(S_norm, nu)

        # Liste des harmoniques
        freq_list = [f0 * k for k in range(1, H+1)]

        freq_peaks, amp_peaks = san.detect_pics(S, freq_list, nu)

        vec = np.zeros(H+1)
        vec[0] = np.log10(np.max(S))
        vec[1:] = np.log10(amp_peaks)
        signaux.append(vec)


    k = np.arange(H+1)
    vec_moy = np.mean(signaux, axis=0)
    Vvec_moy[j] = vec_moy

    k = list(range(H+1))
    indice = []
    ampl = []
    labels = []

    for i in range(5):
        indice.extend(k)
        ampl.extend(signaux[i])
        labels.extend([f"Son {i+1}"] * (H+1))

    indice.extend(k)
    ampl.extend(vec_moy)
    labels.extend(["Moyenne"] * (H+1))

    fig = px.line(x=indice, y=ampl, color=labels, markers=True,title=f"Voyelle: {lettre} - Spectres (log10) des 5 sons + vecteur moyen", labels={"x": "Indice d'harmonique k", "y": "Amplitude log10"}).show()

print(Vvec_moy)

[array([2.64557343, 2.62670417, 2.51545572, 1.93272867, 2.13913407]), array([2.8533247 , 2.77255287, 2.77140889, 2.32289963, 2.59869447]), array([3.06390924, 3.02719011, 2.91139307, 1.94860153, 1.21921098]), array([3.06340046, 2.8252933 , 3.00909505, 2.97563567, 2.41575832]), array([3.18863392, 3.18552527, 2.98740954, 2.0020366 , 1.32035831])]


In [26]:
dat = np.load(f'data_partie_2/test/a_6.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
s = dat['s'] # extraction du signal utile
t = dat['t'] # extraction du vecteur temps associé au signal

s_centre= s - np.mean(s)
nu, S_norm = san.trans_fourier(s_centre, t)
S_norm = np.abs(S_norm)
S_norm /= np.max(S_norm)

f0 = san.detect_fondamentale(S_norm, nu)

# Liste des harmoniques
freq_list = [f0 * k for k in range(1, H+1)]

freq_peaks, amp_peaks = san.detect_pics(S, freq_list, nu)

vec = np.zeros(H+1)
vec[0] = np.log10(np.max(S))
vec[1:] = np.log10(amp_peaks)

def norme(s1,s2):
        return sum(abs(s1-s2))

L_norme = []
    
for i in range(len(lettres)):
    L_norme.append(norme(vec, Vvec_moy[i]))

print(L_norme)
print(min(L_norme))

[np.float64(5.433124017715454), np.float64(6.476905965805054), np.float64(4.9071612596511835), np.float64(7.027056694030762), np.float64(5.171370613574982)]
4.9071612596511835
